In [62]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
matplotlib.style.use('ggplot')

from pymongo import MongoClient
import pprint

# pretty printer
pp = pprint.PrettyPrinter(indent=4)
# setup connector to mongodb
client = MongoClient()

In [82]:
# function to strip "ObjectId()" text from string in project table
def strip_ObjectId(string):
    return string.replace('ObjectId(','').replace(')','')

# load project table and strip objectid text
project_df = pd.read_csv('/srv/zooniverse/tables/mongo_project_table_03-02-17.csv')
project_df['_id'] = project_df['_id'].apply(strip_ObjectId)

In [95]:
result_df = pd.DataFrame()

query = client['talk']['discussions'].find()
for document in query:
    for comment in document['comments']:
        row = {
            'body':comment['body'],
            'created_at':comment['created_at'],
            'mentions':comment['mentions'],
            'tags':comment['tags'],
            'user_name':comment['user_name'],
            'user_id':comment['user_zooniverse_id'],
            'focus_id':document['focus']['_id'],
            'focus_base_type':document['focus']['base_type'],
            'focus_name':document['focus']['name'],
            'focus_type':document['focus']['type'],
            'zooniverse_project_id':project_df.loc[project_df['_id'] == str(document['project_id']), 'zooniverse_id'].values[0],
            'panoptes_project_id':project_df.loc[project_df['_id'] == str(document['project_id']), 'panoptes_id'].values[0]
        }
        if document['board']:
            row['board_id'] = document['board']['_id']
            row['board_name'] = document['board']['name']
            row['board_category'] = document['board']['category']
        else:
            row['board_id'] = None
            row['board_name'] = None
            row['board_category'] = None
        result_df = result_df.append(pd.DataFrame([row]))
result_df

CursorNotFound: Cursor not found, cursor id: 48168688790

,_id,activated_subjects_at,classification_count,complete_count,created_at,display_name,name,panoptes_id,site_prefix,user_count,zooniverse_id
3,ObjectId(502a90cd516bcb060c000001),2017-01-30T08:00:45.339Z,24920000,763694.0,2012-09-10T18:40:05.260Z,Galaxy Zoo,galaxy_zoo,21.0,GZ,152302.0,1
